<a href="https://colab.research.google.com/github/vezvaeia/CS886/blob/master/DataPreparationForTFT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install and Import libraries and define parameters

### Install & Import Libraries

In [ ]:
!pip install transformers
!pip install pytorch-forecasting
!pip install pytorch_lightning

     |████████████████████████████████| 2.1MB 19.3MB/s 
     |████████████████████████████████| 3.3MB 47.2MB/s 
     |████████████████████████████████| 901kB 55.2MB/s 
     |████████████████████████████████| 112kB 19.1MB/s 
     |████████████████████████████████| 296kB 34.9MB/s 
     |████████████████████████████████| 849kB 55.4MB/s 
     |████████████████████████████████| 22.3MB 1.3MB/s 
     |████████████████████████████████| 112kB 60.0MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 163kB 61.3MB/s 
     |████████████████████████████████| 829kB 55.7MB/s 
     |████████████████████████████████| 276kB 53.2MB/s 
     |████████████████████████████████| 276kB 61.4MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 112kB 61.3MB/s 
     |████████████████████████████████| 143kB 59.4MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
  Created wheel for future: filename=future-0.18.2-c

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.functional import softmax
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import csv 
import random
from transformers import BertTokenizer, RobertaTokenizer
from transformers import BertForNextSentencePrediction, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from scipy import stats
import time
import datetime
import pickle
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import gc
import itertools
import nltk
import transformers
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
# nltk.download('punkt')

### Setting device, random seed, and runtime parameters

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Cuda available: ",torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current device: ",  torch.cuda.current_device())

seed = 204920
seed2 = 293652

random.seed(seed2)
np.random.seed(seed2)
torch.manual_seed(seed2)

if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)

Cuda available:  True
Current device:  0


### Setting Parameters

In [ ]:
COMMENT_DIMENSIONS = 20
first_day = datetime.datetime(2020, 1, 1)
last_day = datetime.datetime(2021, 2, 27)
number_of_days = (last_day - first_day).days + 1
print('Number of days: ', number_of_days)

Number of days:  424


# 2. Reading and Processing Datasets

### Moving datasets to Colab 

In [ ]:
!cp  /content/drive/MyDrive/DL-project/Combined.xlsx Combined.xlsx
!cp  /content/drive/MyDrive/DL-project/acaps_covid19_government_measures_dataset_0.xlsx acaps_covid19_government_measures_dataset_0.xlsx
!cp /content/drive/MyDrive/DL-project/time_series_covid_19_confirmed_aggregated.csv time_series_covid_19_confirmed_aggregated.csv
!cp /content/drive/MyDrive/DL-project/comment-embeddings.json comment-embeddings.json
!cp /content/drive/MyDrive/DL-project/reduced-embeddings.json reduced-embeddings.json

### Reading and processing time-series dataset

In [ ]:
time_series_country_list = []
time_series_date_list = [0]
time_series_confirmed_cases = {}

with open('time_series_covid_19_confirmed_aggregated.csv') as csvfile:    
    reader = csv.reader(csvfile, delimiter=',')
    cnt = 0
    
    for row in reader:
        if cnt == 0:
            for i in range(1, len(row)):
                datetimeObject = datetime.datetime.strptime(row[i], '%m/%d/%Y')
                time_series_date_list.append((datetimeObject - first_day).days)
        else:
            country = row[0]
            time_series_country_list.append(country)
            time_series_confirmed_cases[country] = [0 for i in range(number_of_days)]

            previous_accumulative_cases = 0
            for i in range(1, len(row)):
                time_series_confirmed_cases[country][time_series_date_list[i]] = int(row[i]) - previous_accumulative_cases
                previous_accumulative_cases = int(row[i])

        cnt += 1


print('Countries: ', len(time_series_country_list), time_series_country_list)
print('Dates: ', len(time_series_date_list), time_series_date_list)
print('\nConfirmed cases in China: \n', time_series_confirmed_cases['China'])

Countries:  201 ['Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands', 'United Kingdom', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'H

### Reading regulation dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder as le

xl_file = pd.ExcelFile('acaps_covid19_government_measures_dataset_0.xlsx')

sheets = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

dataframe = sheets['Dataset']

print('Sheets in the dataset:   ', sheets.keys())
print('Number of regulations in the dataset: ', len(dataframe), '\n')

# print(dataframe.head())
print('\n', dataframe.info(), '\n')

dataset = {}
for key in dataframe:
    # dataframe[key] = le.fit_transform(dataframe[key].astype(str))
    dataframe[key]=dataframe[key].astype('str')
    dataset[key] = list(dataframe[key])


Sheets in the dataset:    dict_keys(['About', 'Dataset', 'Dictionary'])
Number of regulations in the dataset:  23923 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23923 entries, 0 to 23922
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  23923 non-null  int64         
 1   ISO                 23923 non-null  object        
 2   COUNTRY             23923 non-null  object        
 3   REGION              23923 non-null  object        
 4   ADMIN_LEVEL_NAME    3682 non-null   object        
 5   PCODE               0 non-null      float64       
 6   LOG_TYPE            23923 non-null  object        
 7   CATEGORY            23923 non-null  object        
 8   MEASURE             23923 non-null  object        
 9   TARGETED_POP_GROUP  7556 non-null   object        
 10  COMMENTS            23799 non-null  object        
 11  NON_COMPLIANCE      22764 non-null  obj

### Processing regulation dataset

In [ ]:
good_countries = ['Argentina', 'Austria', 'Bangladesh', 'Belgium', 'Brazil',
       'Canada', 'Chile', 'Colombia', 'Czech Republic', 'France',
       'Germany', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Italy',
       'Japan', 'Mexico', 'Morocco', 'Netherlands', 'Pakistan', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia',
       'South Africa', 'Spain', 'Sweden', 'Switzerland', 'Turkey',
       'Ukraine', 'United Kingdom', 'United States']

In [ ]:
country_list = set([])

for i in range(len(dataset['ID'])):
    country = dataset['COUNTRY'][i]    
    country_list.add(country)
    
country_list = list(country_list)
country_list.sort()
time_series_country_list.sort()

print('Countries in regulation dataset: ', len(country_list), country_list)
print('Countries in time-series dataset: ', len(time_series_country_list), time_series_country_list)
print('Good Countries: ', len(good_countries), good_countries)

common_countries = []
for country in country_list:
    if country in time_series_country_list and country in good_countries:
        common_countries.append(country)

print('Number of countries in intersection of all 3 datasets: ', len(common_countries), common_countries)

regulations = {}

for country in common_countries:
    regulations[country] = [[] for i in range(number_of_days)]

for i in range(len(dataset['ID'])):
    
    # Ignore the regulations with unknown implementation time
    if dataset['DATE_IMPLEMENTED'][i] == 'NaT':
        continue
    
    # Ignore the regulations that we don't have time-series of their countries
    if dataset['COUNTRY'][i] not in common_countries:
        continue

    # Ignore bad countries!
    if dataset['COUNTRY'][i] not in good_countries:
        continue
    
    # fill the empty comment field with ----
    if dataset['COMMENTS'][i] == 'nan':
        dataset['COMMENTS'][i] = '----'


    country = dataset['COUNTRY'][i]
    datetimeObject = datetime.datetime.strptime(dataset['DATE_IMPLEMENTED'][i], '%Y-%m-%d')
    date = (datetimeObject - first_day).days

    regulations[country][date].append(i)
    
print('\nA sample of ids in Iran:')
print(regulations['Iran'])


Countries in regulation dataset:  193 ['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'CAR', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', "CÃ´te d'Ivoire", 'DPRK', 'DRC', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', '

### Creating Measure and Category dictionaries

In [ ]:
measure_dic = {}
category_dic = {}

for i in range(len(dataset['ID'])):
    category = dataset['CATEGORY'][i]
    measure = dataset['MEASURE'][i]

    if category not in category_dic:
        category_dic[category] = 0

    if measure not in measure_dic:
        measure_dic[measure] = 0
    
print(len(category_dic.keys()), category_dic)
print(len(measure_dic.keys()), measure_dic)

6 {'Public health measures': 0, 'Movement restrictions': 0, 'Governance and socio-economic measures': 0, 'Social distancing': 0, 'Lockdown': 0, 'Humanitarian exemption': 0}
35 {'Awareness campaigns': 0, 'Health screenings in airports and border crossings': 0, 'International flights suspension': 0, 'Border checks': 0, 'Strengthening the public health system': 0, 'Isolation and quarantine policies': 0, 'Emergency administrative structures activated or established': 0, 'Surveillance and monitoring': 0, 'Other public health measures enforced': 0, 'Border closure': 0, 'General recommendations': 0, 'State of emergency declared': 0, 'Domestic travel restrictions': 0, 'Limit public gatherings': 0, 'Limit product imports/exports': 0, 'Schools closure': 0, 'Partial lockdown': 0, 'Changes in prison-related policies': 0, 'Closure of businesses and public services': 0, 'Checkpoints within the country': 0, 'Economic measures': 0, 'Military deployment': 0, 'Curfews': 0, 'Visa restrictions': 0, 'Requi

### Creating country2region dictionary

In [ ]:
country2region = {}

for i in range(len(dataset['ID'])):
    country = dataset['COUNTRY'][i]
    region = dataset['REGION'][i]

    if country not in country2region:
        country2region[country] = region

print(country2region)

{'Afghanistan': 'Asia', 'Albania': 'Europe', 'Algeria': 'Africa', 'Angola': 'Africa', 'Antigua and Barbuda': 'Americas', 'Argentina': 'Americas', 'Armenia': 'Middle east', 'Australia': 'Pacific', 'Austria': 'Europe', 'Azerbaijan': 'Middle east', 'Bahamas': 'Americas', 'Bahrain': 'Middle east', 'Bangladesh': 'Asia', 'Barbados': 'Americas', 'Belarus': 'Europe', 'Belgium': 'Europe', 'Belize': 'Americas', 'Benin': 'Africa', 'Bhutan': 'Middle east', 'Bolivia': 'Americas', 'Bosnia and Herzegovina': 'Europe', 'Botswana': 'Africa', 'Brazil': 'Americas', 'Brunei': 'Middle east', 'Bulgaria': 'Europe', 'Burkina Faso': 'Africa', 'Burundi': 'Africa', "CÃ´te d'Ivoire": 'Africa', 'Cambodia': 'Asia', 'Cameroon': 'Africa', 'Canada': 'Americas', 'Cape Verde': 'Africa', 'CAR': 'Africa', 'Chad': 'Africa', 'Chile': 'Americas', 'China': 'Asia', 'Colombia': 'Americas', 'Comoros': 'Africa', 'Congo': 'Africa', 'Costa Rica': 'Americas', 'Croatia': 'Europe', 'Cuba': 'Americas', 'Cyprus': 'Europe', 'Czech Republi

# 3. Comment embeddings from DeBerta model

### creating list of comments

In [ ]:
comments = []
comment_infos = []

for country in regulations:
    for day in range(len(regulations[country])):

        if len(regulations[country][day]) == 0:
                continue

        comment = ''
        for index in regulations[country][day]:
            comment = comment + ' ' + dataset['COMMENTS'][index]
            
        comments.append(comment)
        comment_infos.append((country, day))

print('Number of comments: ', len(comments))
print(comments[0:5])
print(comment_infos[0:5])

Number of comments:  2849
[' Campaign for argentinians abroad', ' For long term ', ' Flights from countries with COVID transmissions. From Europe, UK, USA, South Korea, Japan, China and Iran, for 14 days Restrict public gathering for sport events and closes cultural spaces', ' For non-residents and travellers from the Schengen Area, UK, US, Iran, China, Japan and South Corea', ' Social protection plan to ensure alimentation and health for all All relevant public services transfered online. For 2 weeks.']
[('Argentina', 47), ('Argentina', 70), ('Argentina', 71), ('Argentina', 74), ('Argentina', 75)]


### Tokenizing

In [ ]:
tokenizer = transformers.DebertaTokenizer.from_pretrained('microsoft/deberta-base') 
max_length = 256
train_encodings = tokenizer(comments, add_special_tokens=True, return_token_type_ids=False, truncation=True, padding=True, max_length=max_length)

### Creating pytorch Dataset

In [ ]:
class NSPDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
#         self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels != None:
#           item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # return len(self.labels)
        return len(self.encodings['input_ids'])

In [ ]:
train_dataset = NSPDataset(train_encodings)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

### Creating DeBerta Model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = transformers.DebertaModel.from_pretrained('microsoft/deberta-base')


if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
  model = torch.nn.DataParallel(model)
    
model.to(device)
model.train()

DebertaModel(
  (embeddings): DebertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=0)
    (LayerNorm): DebertaLayerNorm()
    (dropout): StableDropout()
  )
  (encoder): DebertaEncoder(
    (layer): ModuleList(
      (0): DebertaLayer(
        (attention): DebertaAttention(
          (self): DisentangledSelfAttention(
            (in_proj): Linear(in_features=768, out_features=2304, bias=False)
            (pos_dropout): StableDropout()
            (pos_proj): Linear(in_features=768, out_features=768, bias=False)
            (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
            (dropout): StableDropout()
          )
          (output): DebertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): DebertaLayerNorm()
            (dropout): StableDropout()
          )
        )
        (intermediate): DebertaIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bia

### Generating Comment Embeddings

In [ ]:
all_cls = []

for iteration, batch in tqdm(enumerate(train_loader)):
    
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    
    with torch.no_grad():
        
        last_hidden_state = model(input_ids, attention_mask=attention_mask).last_hidden_state
        cls_tokens = last_hidden_state[:,0,:].detach()
        
        all_cls.append(cls_tokens)

        
# out_cls is a matrix of size number_of_not_null_comments (23799) X size_of_hidden_state_of_BERT (768)
# In this matrix, for each comment we have an embedding vector.
out_cls = torch.cat(all_cls, 0)

print("shape of output matrix :", out_cls.shape)
comment_embeddings = out_cls.tolist()

90it [01:00,  1.48it/s]

shape of output matrix : torch.Size([2849, 768])


### Reduce embeddings

In [ ]:
reduced_embeddings = comment_embeddings
print(len(reduced_embeddings), len(reduced_embeddings[0]), reduced_embeddings[0])
COMMENT_DIMENSIONS = len(reduced_embeddings[0])
print(COMMENT_DIMENSIONS)

2849 768 [0.05790683627128601, -0.020389284938573837, -0.05915236845612526, -0.038918204605579376, 0.06387216597795486, 0.08327605575323105, -0.09359509497880936, -0.0012470055371522903, 0.10593076050281525, -0.022725239396095276, -0.035745494067668915, 0.01918317936360836, 0.00011213100515305996, -0.014184052124619484, -0.008730756118893623, -0.03663644939661026, -0.002207149751484394, 0.04941564053297043, 0.06737002730369568, -0.004412967711687088, 0.08805371820926666, 0.06810662150382996, 0.042076997458934784, -0.08858568966388702, 0.04747886210680008, 0.010467100888490677, 0.06681830435991287, 0.08818880468606949, 0.0038609355688095093, 0.008509274572134018, -0.05409366264939308, 0.03661562502384186, 0.01395716518163681, -0.0194980688393116, 0.09344112128019333, 0.03765181452035904, 0.0076604485511779785, 0.004985209554433823, 0.055208154022693634, 0.08489901572465897, -0.008693907409906387, -0.02737007476389408, -0.03662710636854172, 0.08753152936697006, -0.042322658002376556, 0.0

### Move embeddings to dictionary

In [ ]:
info2embedding = {}
for i in range(len(reduced_embeddings)):
    info2embedding[comment_infos[i]] = reduced_embeddings[i]

for key in info2embedding:
    print(key, ':\n', info2embedding[key])
    break

('Argentina', 47) :
 [0.05790683627128601, -0.020389284938573837, -0.05915236845612526, -0.038918204605579376, 0.06387216597795486, 0.08327605575323105, -0.09359509497880936, -0.0012470055371522903, 0.10593076050281525, -0.022725239396095276, -0.035745494067668915, 0.01918317936360836, 0.00011213100515305996, -0.014184052124619484, -0.008730756118893623, -0.03663644939661026, -0.002207149751484394, 0.04941564053297043, 0.06737002730369568, -0.004412967711687088, 0.08805371820926666, 0.06810662150382996, 0.042076997458934784, -0.08858568966388702, 0.04747886210680008, 0.010467100888490677, 0.06681830435991287, 0.08818880468606949, 0.0038609355688095093, 0.008509274572134018, -0.05409366264939308, 0.03661562502384186, 0.01395716518163681, -0.0194980688393116, 0.09344112128019333, 0.03765181452035904, 0.0076604485511779785, 0.004985209554433823, 0.055208154022693634, 0.08489901572465897, -0.008693907409906387, -0.02737007476389408, -0.03662710636854172, 0.08753152936697006, -0.04232265800

# 4. Creating Pandas dataframe

In [ ]:
features = {'time_idx': [], 'country': [], 'region': [], 'confirmed-cases': []}

for i in range(COMMENT_DIMENSIONS):
    features['comment-embedding' + str(i)] = []

dataframe = pd.DataFrame(features)

for country in tqdm(regulations):
    for day in range(number_of_days):

        new_entry = {}
        new_entry['country'] = country
        new_entry['time_idx'] = day
        new_entry['region'] = country2region[country]
        new_entry['confirmed-cases'] = time_series_confirmed_cases[country][day]
            
        embedding = [0 for i in range(COMMENT_DIMENSIONS)]
        if (country, day) in info2embedding:
            embedding = info2embedding[(country, day)]

        for i in range(COMMENT_DIMENSIONS):
            new_entry['comment-embedding' + str(i)] = embedding[i]
        
        dataframe = dataframe.append(new_entry, ignore_index=True)

print(dataframe.info())
dataframe.to_excel('data2.xlsx', sheet_name='main', index=False)
dataframe.to_excel('/content/drive/MyDrive/DL-project/data2.xlsx', sheet_name='main', index=False)




  0%|          | 0/37 [00:00<?, ?it/s]


  3%|▎         | 1/37 [00:34<20:44, 34.56s/it]


  5%|▌         | 2/37 [01:10<20:21, 34.89s/it]


  8%|▊         | 3/37 [01:46<20:03, 35.40s/it]


 11%|█         | 4/37 [02:24<19:50, 36.07s/it]


 14%|█▎        | 5/37 [03:03<19:41, 36.91s/it]


 16%|█▌        | 6/37 [03:42<19:29, 37.73s/it]


 19%|█▉        | 7/37 [04:24<19:23, 38.78s/it]


 22%|██▏       | 8/37 [05:05<19:08, 39.61s/it]


 24%|██▍       | 9/37 [05:49<19:00, 40.74s/it]


 27%|██▋       | 10/37 [06:33<18:50, 41.87s/it]


 30%|██▉       | 11/37 [07:19<18:42, 43.16s/it]


 32%|███▏      | 12/37 [08:06<18:24, 44.17s/it]


 35%|███▌      | 13/37 [08:54<18:07, 45.32s/it]


 38%|███▊      | 14/37 [09:43<17:48, 46.46s/it]


 41%|████      | 15/37 [10:34<17:33, 47.89s/it]


 43%|████▎     | 16/37 [11:28<17:22, 49.65s/it]


 46%|████▌     | 17/37 [12:22<16:58, 50.94s/it]


 49%|████▊     | 18/37 [13:17<16:33, 52.26s/it]


 51%|█████▏    | 19/37 [14:15<16:07, 53.78s/it]


 54%|█████▍    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15688 entries, 0 to 15687
Columns: 772 entries, time_idx to comment-embedding767
dtypes: float64(770), object(2)
memory usage: 92.4+ MB
None


# 5. Testing TFT usage

### Reading data file

In [ ]:
xl_file = pd.ExcelFile('data.xlsx')

sheets = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

data = sheets['main']

print('\n', dataframe.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74200 entries, 0 to 74199
Columns: 126 entries, time_idx to negative-comment-embedding19
dtypes: float64(124), object(2)
memory usage: 71.3+ MB

 None 



### Entering training data

In [ ]:
max_prediction_length = 15
max_encoder_length = 60
training_cutoff = data["time_idx"].max() - max_prediction_length

feature_list = []
for log_type in ['positive', 'negative']:
    for category in category_dic:
        feature_list.append(log_type + '-' + category)
    for measure in measure_dic:
        feature_list.append(log_type + '-' + measure) 
    for i in range(COMMENT_DIMENSIONS):
        feature_list.append(log_type + '-comment-embedding' + str(i)) 

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="confirmed-cases",
    group_ids=["country"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["region"],
    static_reals=[],
    time_varying_known_categoricals=[],  # We can further add information like "special_days" and "month"
    variable_groups={},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=feature_list,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

/usr/local/lib/python3.7/dist-packages/pytorch_forecasting/data/timeseries.py:517: UserWarning: Target scales will be only added for continous targets
  warnings.warn("Target scales will be only added for continous targets", UserWarning)


KeyError: ignored